In [1]:
!pip install praw

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from praw import reddit
import json

In [3]:
import praw

reddit = praw.Reddit(
    client_id="LtjnfEAH_rRdFDbMwILwxw",
    client_secret="04WSBZHCzqm0ElLYMRkmTkmijCCSog",
    password="PosterThreading69",
    user_agent="my user agent",
    username="postthreadbot",
)

Version 7.5.0 of praw is outdated. Version 7.6.0 was released Tuesday May 10, 2022.


In [4]:
subreddits = "announcements+Art+AskReddit+askscience+atheism+aww+blog+books+creepy+dataisbeautiful+DIY+Documentaries+EarthPorn+explainlikeimfive+food+funny+Futurology+gadgets+gaming+GetMotivated+gifs+history+IAmA+InternetIsBeautiful+Jokes+LifeProTips+listentothis+mildlyinteresting+movies+Music+news+nosleep+nottheonion+OldSchoolCool+personalfinance+philosophy+photoshopbattles+pics+science+Showerthoughts+space+sports+television+tifu+todayilearned+TwoXChromosomes+UpliftingNews+videos+worldnews+WritingPrompts"

In [5]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="ws://35.80.10.26:11946",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [104]:
# Function to make extrinsic calls
def make_call(call_module, call_function, call_params, keypair):
    call = substrate.compose_call(
        call_module=call_module,
        call_function=call_function,
        call_params=call_params
    )

    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair)

    try:
        receipt = substrate.submit_extrinsic(extrinsic, wait_for_inclusion=True, wait_for_finalization=True)
        print("Extrinsic '{}' sent and included in block '{}'".format(receipt.extrinsic_hash, receipt.block_hash))

    except SubstrateRequestException as e:
        print("Failed to send: {}".format(e))
    return receipt

In [7]:
bob = Keypair.create_from_uri('//Bob')

In [45]:
schema = "subreddit,author,title,selftext,url,is_nsfw,blah"

schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

schemaId = -1
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if schemaTemp == schema:
        schemaId = i
        print(schemaTemp.value)
        break
    
if schemaId == -1:
    receipt = make_call("Schemas", "register_schema", {"schema": schema}, bob)
    for event in receipt.triggered_events:
        event = event.decode()
        if event['event']['module_id'] == 'SchemaRegistered':
            schemaId = event['event']['attributes'][1]

print(schemaId)

subreddit,author,title,selftext,url,is_nsfw,blah
5


In [46]:
msa_key = substrate.query(
    module='Msa',
    storage_function='KeyInfoOf',
    params=[bob.ss58_address]
)

if msa_key == None:
    make_call("Msa", "create", {}, bob)
    msa_key = substrate.query(
        module='Msa',
        storage_function='KeyInfoOf',
        params=[bob.ss58_address]
    )
    
msa_key

<scale_info::245(value={'msa_id': 4, 'nonce': 0, 'expired': 0})>

In [105]:
i = 0
for comment in reddit.subreddit('all').stream.submissions():
    new_post = f"{str(comment.subreddit)},{str(comment.author)},{str(comment.title)},{str(comment.selftext)},{str(comment.url)},{comment.over_18}"
    call_params = {
        "schema_id": schemaId,
        "message": new_post
    }
    receipt = make_call("Messages", "add", call_params, bob)
    i += 1
    if i > 0:
        break

{'call': '0x2300050061017361766564666f7264696c6c752c69637269737469616e6872696d6975632c53686172696e6720697320636172696e672c2c68747470733a2f2f692e696d6775722e636f6d2f5161416f3967522e676966762c46616c7365', 'era': '00', 'nonce': 29, 'tip': 0, 'spec_version': 1, 'genesis_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'block_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'transaction_version': 1, 'asset_id': {'tip': 0, 'asset_id': None}} call CallBytes b'#\x00\x05\x00a\x01savedfordillu,icristianhrimiuc,Sharing is caring,,https://i.imgur.com/QaAo9gR.gifv,False' 0x2300050061017361766564666f7264696c6c752c69637269737469616e6872696d6975632c53686172696e6720697320636172696e672c2c68747470733a2f2f692e696d6775722e636f6d2f5161416f3967522e676966762c46616c7365

{'call': '0x2300050061017361766564666f7264696c6c752c69637269737469616e6872696d6975632c53686172696e6720697320636172696e672c2c68747470733a2f2f692e696d6775722e636f6d2f5161416f3967522e6769667

Extrinsic '0x9e9d655a7dc5e613edda1cd2894000306f1221489b112d9ea017b94d991ddb84' sent and included in block '0x52c98c41f53e2ad11aeca04c44a5642005706c5caebe9028597c123f8534955f'


In [68]:
substrate.get_metadata_storage_functions()

[{'storage_name': 'Account',
  'storage_modifier': 'Default',
  'storage_default_scale': bytearray(b'A\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
  'storage_default': {'nonce': 0,
   'consumers': 0,
   'providers': 0,
   'sufficients': 0,
   'data': {'free': 0, 'reserved': 0, 'misc_frozen': 0, 'fee_frozen': 0}},
  'documentation': ' The full account information for a particular account ID.',
  'module_id': 'System',
  'module_prefix': 'System',
  'module_name': 'System',
  'spec_version': 1,
  'type_keys': ['scale_info::0'],
  'type_hashers': ['Blake2_128Concat'],
  'type_value': 'scale_info::3',
  'type_class': 'Map'},
 {'storage_name': 'ExtrinsicCount',
  'storage_modifier': 'Optional',
  'storage_default

In [106]:
receipt.error_message

In [107]:
receipt.triggered_events

[<scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a08', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 1984426576)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 1984426576), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a07', 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 2461399)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 2461399), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0000', 'module_id': 'System', 'event_id': 'ExtrinsicSuccess', 'attributes': {'weight': 125088000, 'class': 'Norma

In [110]:
receipt.block_number

In [91]:
msa_key.decode()['msa_id']

4

In [119]:
receipt.triggered_events

[<scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a08', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 1984426576)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 1984426576), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a07', 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 2461399)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 2461399), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0000', 'module_id': 'System', 'event_id': 'ExtrinsicSuccess', 'attributes': {'weight': 125088000, 'class': 'Norma

In [96]:
substrate.query(
    module='Messages',
    storage_function='Messages',
    params=[4810,msa_key.decode()['msa_id']+1]
)

<scale_info::254(value=[{'data': 'ask,Ki_no_Shirayuki,Should downvoting be removed?,,https://www.reddit.com/r/ask/comments/v6mwxb/should_downvoting_be_removed/,False', 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 'msa_id': 4, 'index': 0}, {'data': 'AskReddit,Evening-Glass-8152,What was the greatest disappointment that happened during sex for you?,,https://www.reddit.com/r/AskReddit/comments/v6mwxg/what_was_the_greatest_disappointment_that/,True', 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 'msa_id': 4, 'index': 1}, {'data': 'Mexico_Economy,bot_painani,INEGI Noticias: Indicador Global de la Actividad Económica,,https://www.inegi.org.mx/app/saladeprensa/noticia.html?id=3789,False', 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty', 'msa_id': 4, 'index': 2}, {'data': 'RandomThoughts,Crystal_crown1,pride,If pride month didnt exist, there probably would be less gay people.,https://www.reddit.com/r/RandomThoughts/comments/v6mwxk/pride/,False', 's

In [120]:
substrate.query(
    module='Messages',
    storage_function='BlockMessages',
    params=[]
)

<scale_info::248(value=[])>